In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
import re
import string
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [2]:
data = pd.read_csv('SearchResults.csv')
cols_to_drop = ['Rank', 'NCT Number', 'Acronym', 'Status', 'Study Results',
         'Outcome Measures',
       'Sponsor/Collaborators', 'Gender', 'Age', 'Enrollment',
       'Funded Bys', 'Study Type', 'Study Designs', 'Other IDs', 'Start Date',
       'Primary Completion Date', 'Completion Date', 'First Posted',
       'Results First Posted', 'Last Update Posted', 'Locations',
       'Study Documents', 'URL']

for col in cols_to_drop:
    data = data.drop([col],axis=1)
data.head()

,Title,Conditions,Interventions,Phases
0,Study of SD-101 in Combination With Localized ...,B-cell Lymphoma,Drug: SD-101|Radiation: Radiation therapy,Phase 1|Phase 2
1,Bruton's Tyrosine Kinase (BTK) Inhibition in B...,B-cell Lymphoma,Drug: Ibrutinib and immunochemotherapies,Phase 1
2,A Study Evaluating CPI-1205 in Patients With B...,B-Cell Lymphoma,Drug: CPI-1205,Phase 1
3,Treatment of Mature B-cell Lymphoma/Leukaemia,B-Cell Lymphoma,Drug: half cyclophosphamide|Drug: without COPA...,Phase 3
4,"INCB050465 in Combination With Rituximab, Bend...",B-cell Lymphoma,Drug: INCB050465|Drug: Rituximab|Drug: Bendamu...,Phase 1


In [3]:
def remove_punctuations_from_string(string1):
    string1 = string1.lower() #changing to lower case
    translation_table = dict.fromkeys(map(ord, string.punctuation), ' ') #creating dictionary of punc & None
    string2 = string1.translate(translation_table) #translating string1
    return string2

def remove_stopwords_from_string(string1):
    pattern = re.compile(r'\b(' + r'|'.join(nltk.corpus.stopwords.words('english')) + r')\b\s*') #compiling all stopwords.
    string2 = pattern.sub('', string1) #replacing the occurrences of stopwords in string1
    return string2

In [4]:
data['Interventions'] = data['Interventions'].astype(str)
data['Title'] = data['Title'].astype(str)

In [5]:
data['Interventions'] = data['Interventions'].apply(lambda x:remove_punctuations_from_string(x))
data['Interventions'] = data['Interventions'].apply(lambda x:remove_stopwords_from_string(x))
data['Title'] = data['Title'].apply(lambda x:remove_punctuations_from_string(x))
data['Title'] = data['Title'].apply(lambda x:remove_stopwords_from_string(x))
data.head()

,Title,Conditions,Interventions,Phases
0,study sd 101 combination localized low dose ra...,B-cell Lymphoma,drug sd 101 radiation radiation therapy,Phase 1|Phase 2
1,bruton tyrosine kinase btk inhibition b cell...,B-cell Lymphoma,drug ibrutinib immunochemotherapies,Phase 1
2,study evaluating cpi 1205 patients b cell lymp...,B-Cell Lymphoma,drug cpi 1205,Phase 1
3,treatment mature b cell lymphoma leukaemia,B-Cell Lymphoma,drug half cyclophosphamide drug without copa...,Phase 3
4,incb050465 combination rituximab bendamustine...,B-cell Lymphoma,drug incb050465 drug rituximab drug bendamu...,Phase 1


In [6]:
data['Phases'] = data['Phases'].astype(str)

In [7]:
data['Phases'] = data['Phases'].apply(lambda x:remove_punctuations_from_string(x))
data['Phases'] = data['Phases'].apply(lambda x:remove_stopwords_from_string(x))
data.head()

,Title,Conditions,Interventions,Phases
0,study sd 101 combination localized low dose ra...,B-cell Lymphoma,drug sd 101 radiation radiation therapy,phase 1 phase 2
1,bruton tyrosine kinase btk inhibition b cell...,B-cell Lymphoma,drug ibrutinib immunochemotherapies,phase 1
2,study evaluating cpi 1205 patients b cell lymp...,B-Cell Lymphoma,drug cpi 1205,phase 1
3,treatment mature b cell lymphoma leukaemia,B-Cell Lymphoma,drug half cyclophosphamide drug without copa...,phase 3
4,incb050465 combination rituximab bendamustine...,B-cell Lymphoma,drug incb050465 drug rituximab drug bendamu...,phase 1


In [8]:
data = data.drop(['Conditions'],axis=1)

In [14]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/irochka/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [17]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
data['Title'] = [stemmer.stem(word) for word in data['Title'].astype(str)]
data['Title'] = [lemmatizer.lemmatize(word) for word in data['Title'].astype(str)]
data['Interventions'] = [stemmer.stem(word) for word in data['Interventions'].astype(str)]
data['Interventions'] = [lemmatizer.lemmatize(word) for word in data['Interventions'].astype(str)]
data.head(25)

,Title,Interventions,Phases
0,study sd 101 combination localized low dose ra...,drug sd 101 radiation radiation therapi,phase 1 phase 2
1,bruton tyrosine kinase btk inhibition b cell...,drug ibrutinib immunochemotherapi,phase 1
2,study evaluating cpi 1205 patients b cell lymp...,drug cpi 1205,phase 1
3,treatment mature b cell lymphoma leukaemia,drug half cyclophosphamide drug without copa...,phase 3
4,incb050465 combination rituximab bendamustine...,drug incb050465 drug rituximab drug bendamu...,phase 1
5,evaluation 4th generation safety designed car ...,genetic anti cd19 car cel,phase 1 phase 2
6,prognosis patients aggressive b cell lymphoma ...,nan,nan
7,refractory diffuse large b cell lymphoma,nan,nan
8,bendamustine plus rituximab br relapsed prog...,drug bendamustine plus rituximab,phase 2
9,novel autologou car therapy relapsed refractor...,combination product car,phase 2


In [18]:
data.to_csv('stemmed_data.csv')

In [19]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,15), analyzer='char')

In [20]:
data_tfidf_char_fit = tfidf_vectorizer.fit_transform(data)


In [21]:
data_tfidf_char_fit

<3x113 sparse matrix of type '<class 'numpy.float64'>'
	with 119 stored elements in Compressed Sparse Row format>

In [22]:
data['tf_char'] = data_tfidf_char_fit

In [23]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold,train_test_split

In [24]:
data2 = pd.read_csv('data_processed.csv')
data['KM_label'] = data2['KMeans_24clust']

In [25]:
del data2